# This notebook is to prepare a dataset for training the ANNABELL model.
The dataset is derived from the SQuAD database.  Each Question and Answer pair was used to prompt a LLM to provide a declarative statement.


In [1]:
import pandas as pd
import re
from datasets import load_dataset, load_from_disk
from collections import Counter
from generate_declarative_sentences import load_squad_dataset

# Load the SQuAD dataset
ds = load_squad_dataset()

# Load the datafile into a pandas DataFrame
train_filepath = "datasets/train/declarative_sentences_train.tsv"
validation_filepath = "datasets/validation/declarative_sentences_validation.tsv"
train_df = pd.read_csv(train_filepath, sep="\t")
validation_df = pd.read_csv(validation_filepath, sep="\t")
print(train_df.info())
print(validation_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816 entries, 0 to 815
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 816 non-null    object
 1   title              816 non-null    object
 2   question           816 non-null    object
 3   answer             816 non-null    object
 4   response_question  816 non-null    object
 5   response_answer    816 non-null    object
 6   statement          816 non-null    object
dtypes: object(7)
memory usage: 44.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 0 non-null      object
 1   title              0 non-null      object
 2   question           0 non-null      object
 3   answer             0 non-null      object
 4   response_question  0 non-null      object
 5   r

## To be useable for training, the ANNABELL model,  the examples need to follow the specific format below:
* Uppercase letters are used only for first letter of proper nouns – e.g. Chris, London, Big Ben
* Questions start with a question mark – e.g. "how old are you"
* Words with a suffix are split in the form base –suffix.  e.g. animals -> animal –s, writing \t *> write \t *ing
    * Apart from the above exceptions the following rules apply:
    * every character must be lowercase
    * No punctuation
    * No Special Characters
    * No Whitespace between lines
    * Lines can be prefixed with # to insert comments
    * If .ph is used, the entire phrase in the exact format must be input

In [2]:
#move the ? from the end of each question to the start
def move_question_mark_to_start(question):
    if question.strip().endswith("?"):
        edited_question = "? " + question[:-1]
    else:
        #raise an exception if the question does not end with a ?
        raise ValueError(f"Question does not end with a question mark: {question}")
    return edited_question

In [3]:
def replace_decimal_in_matched_string(matched_string):
    #callable function to support the regex
    number_str = matched_string.group(0)
    return number_str.replace('.', ' point ')

In [4]:
def convert_decimal_point_to_word(a_string):
    import re
    #Replace "." with 'point' if it is part of a number.

    # Regex to find numbers with a decimal point:
    # \d+\.\d+  matches numbers like 1.23
    # \d+\.     matches numbers like 1. (dot at the end after digits)
    # \.\d+     matches numbers like .5 (dot at the beginning before digits)
    # The order matters to match \d+\.\d+ before \d+\. or \.\d+ for overlapping cases.
    pattern = r'\d+\.\d+|\d+\.|\.\d+'
    return re.sub(pattern, replace_decimal_in_matched_string, a_string)

In [5]:
from unidecode import unidecode

def remove_accents(text):
    #Convert accented characters to unaccented ones
    text_unaccented = unidecode(text)
    return text_unaccented

In [6]:
#remove all special characters except question marks and hyphen form the statements
def remove_special_characters(text):
    """
    Removes special characters from a string, keeping alphanumeric characters and spaces.
    """
    # Keep only alphanumeric characters and spaces
    cleaned_text = re.sub(r'[^A-Za-z0-9\s?-]+', '', text)
    return cleaned_text

In [7]:
def filter_by_max_words(the_df, max_words=10):
    #returnes a new dataframe filtered such that each question, answer and statement has less than 11 words

    filtered_df = the_df[the_df.apply(lambda row: len(row["question"].split()) <= max_words and len(row["answer"][0].split()) <= max_words and len(row["statement"].split()) <= max_words , axis=1)]
    return filtered_df

In [8]:
def clean_text(a_series, is_question):
    #takes a dataframe series and applies reformatting
    if is_question:
        a_series = a_series.apply(move_question_mark_to_start)
    for function_name in (convert_decimal_point_to_word, remove_accents, remove_special_characters):
        a_series = a_series.apply(function_name)
    return a_series

In [9]:
filtered_train_df = filter_by_max_words(train_df, max_words=10)
filtered_train_df["response_question"] = clean_text(filtered_train_df["response_question"], True)
filtered_train_df["response_answer"] = clean_text(filtered_train_df["response_answer"], False)
filtered_train_df["statement"] = clean_text(filtered_train_df["statement"], False)
filtered_train_df

/var/folders/cv/z7xz0c3n3nd4c26hj7g01n7w0000gn/T/ipykernel_39427/1758001371.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train_df["response_question"] = clean_text(filtered_train_df["response_question"], True)
/var/folders/cv/z7xz0c3n3nd4c26hj7g01n7w0000gn/T/ipykernel_39427/1758001371.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train_df["response_answer"] = clean_text(filtered_train_df["response_answer"], False)
/var/folders/cv/z7xz0c3n3nd4c26hj7g01n7w0000gn/T/ipykernel_3942

,id,title,question,answer,response_question,response_answer,statement
1,56ce304daab44d1400b8850f,New_York_City,In what city is the United Nations based?,New York,? in what city is the united nations based,new york,the united nations is based in New York
3,56ce304daab44d1400b88511,New_York_City,What American city welcomes the largest number...,New York,? What American city welcomes the largest numb...,new york,new york welcomes the largest number of legal ...
6,56ce3124aab44d1400b8852a,New_York_City,How many boroughs comprise New York City?,five,? How many boroughs comprise New York City,five,New York City comprise -s five borough -s
11,56cf9d81234ae51400d9be1b,New_York_City,How man boroughs does New York City contain?,five,? How man boroughs does New York City contain,five,New York City contains five borough -s
17,56ce31baaab44d1400b8853b,New_York_City,What nation founded New Amsterdam?,the Dutch Republic,? What nation founded New Amsterdam,the Dutch Republic,the Dutch Republic founded New Amsterdam
...,...,...,...,...,...,...,...
790,56d11b4a17492d1400aab994,New_York_City,The Queensboro Bridge utilized what type of co...,cantilever,? the Queensboro Bridge utilized what type of ...,cantilever,the Queensboro Bridge utilized a cantilever co...
796,56d11e7b17492d1400aab9d3,New_York_City,How long is Newtown Creek in kilometers?,6,? How long is Newtown Creek in kilometers,6,newtown creek is 6 kilometers long
798,56d11e7b17492d1400aab9d5,New_York_City,What notable accidental fossil fuel discharge ...,the Greenpoint oil spill,? what notable accidental fossil fuel discharg...,the Greenpoint oil spill,the Greenpoint oil spill occurred at Newtown C...
799,56d11eb317492d1400aab9d9,New_York_City,What type of government does New York City have?,mayor-council,? what type of government does New York City have,mayor-council,new york city has a mayor-council government


In [41]:
#write a file that can be used to train ANNABELL
output_filename = "training/nyc_statements.txt"
with open(output_filename, "w") as output_file:
    for statement in filtered_train_df["statement"]:
        output_file.write(statement + "\n")
#check that the file looks correct
with open(output_file, "r") as input_file:
    lines = input_file.readlines()
print(lines[:5])

In [42]:
with open(output_filename, "r") as input_file:
    lines = input_file.readlines()
print(lines[:5])

['the united nations is based in New York\n', 'new york welcomes the largest number of legal immigrants\n', 'New York City comprise -s five borough -s\n', 'New York City contains five borough -s\n', 'the Dutch Republic founded New Amsterdam\n']


In [23]:
# summary of validation dataset
ds_valid=ds["validation"].to_pandas()
print(ds_valid.info())
ds_valid["title"]
print("number of titles: " + str(len(set(titles))))
print((set(titles)))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10570 entries, 0 to 10569
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        10570 non-null  object
 1   title     10570 non-null  object
 2   context   10570 non-null  object
 3   question  10570 non-null  object
 4   answers   10570 non-null  object
dtypes: object(5)
memory usage: 413.0+ KB
None
number of titles: 48
{'French_and_Indian_War', 'Nikola_Tesla', 'Apollo_program', 'Rhine', 'Construction', 'Geology', 'Southern_California', 'Martin_Luther', 'Black_Death', 'Victoria_(Australia)', 'Steam_engine', 'Scottish_Parliament', 'Ctenophora', 'Islamism', 'European_Union_law', 'Intergovernmental_Panel_on_Climate_Change', 'Teacher', 'Private_school', 'Economic_inequality', 'Force', 'Harvard_University', 'Victoria_and_Albert_Museum', 'Immune_system', 'Prime_number', 'Civil_disobedience', 'Computational_complexity_theory', 'Normans', 'Huguenot', 'Kenya', 'University_of_C

In [24]:
from collections import Counter
bag_of_titles = Counter((titles))
(bag_of_titles.most_common(20))

[('Super_Bowl_50', 810),
 ('Nikola_Tesla', 511),
 ('Martin_Luther', 479),
 ('American_Broadcasting_Company', 407),
 ('Doctor_Who', 319),
 ('Economic_inequality', 297),
 ('Victoria_and_Albert_Museum', 295),
 ('Chloroplast', 295),
 ('Rhine', 291),
 ('Teacher', 268),
 ('Newcastle_upon_Tyne', 264),
 ('Warsaw', 247),
 ('Oxygen', 247),
 ('Genghis_Khan', 245),
 ('European_Union_law', 243),
 ('Apollo_program', 242),
 ('Immune_system', 238),
 ('Kenya', 231),
 ('Yuan_dynasty', 228),
 ('Huguenot', 217)]

In [42]:
#produce a summary of a dataset by splits
def dataset_summary(a_dataset):
    for split in ds.keys():
        ds_split=ds[split].to_pandas()
        print("summary of " + split + " split")
        print(ds_split.info())
        titles=ds_split["title"]
        print("number of titles: " + str(len(set(titles))))
        print((set(titles)))
        bag_of_titles = Counter((titles))
        print("titles with most numerous examples: " + str((bag_of_titles.most_common(20))) +"\n")

In [43]:
dataset_summary(ds)

summary of train split
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87599 entries, 0 to 87598
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        87599 non-null  object
 1   title     87599 non-null  object
 2   context   87599 non-null  object
 3   question  87599 non-null  object
 4   answers   87599 non-null  object
dtypes: object(5)
memory usage: 3.3+ MB
None
number of titles: 442
{'Alps', 'Super_Nintendo_Entertainment_System', 'University_of_Notre_Dame', 'Web_browser', 'Atlantic_City,_New_Jersey', 'Asthma', 'Hokkien', 'Pain', 'Houston', 'Institute_of_technology', 'Molotov%E2%80%93Ribbentrop_Pact', 'House_music', 'Antarctica', 'Namibia', 'Financial_crisis_of_2007%E2%80%9308', 'Humanism', 'Lancashire', 'Presbyterianism', 'Animal', 'Southampton', 'Madonna_(entertainer)', 'Universal_Studios', 'Qing_dynasty', 'High-definition_television', '2008_Summer_Olympics_torch_relay', 'Sony_Music_Entertainment', 'Canon_la